In [1]:
!pip install gym-minesweeper

In [2]:
import gym
import gym_minesweeper

env = gym.make("Minesweeper-v0")  # Default 16x16 board, 40 mines
state = env.reset()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: De

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random

class DQN(nn.Module):
    def __init__(self, obs_shape, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(np.prod(obs_shape), 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Hyperparameters
episodes = 1000
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.995
lr = 0.001
batch_size = 64
memory_size = 10000

obs_shape = env.observation_space.shape
# Calculate action_size for MultiDiscrete space
action_size = np.prod(env.action_space.nvec)


policy_net = DQN(obs_shape, action_size)
target_net = DQN(obs_shape, action_size)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.Adam(policy_net.parameters(), lr=lr)
memory = deque(maxlen=memory_size)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:
def select_action(state, epsilon):
    if np.random.rand() < epsilon:
        # For a Discrete(3) action space, sample returns a single integer (0, 1, or 2)
        return env.action_space.sample()

    # Flatten the state for the neural network
    state = np.array(state).flatten()
    state = torch.FloatTensor(state).unsqueeze(0)
    with torch.no_grad():
        q_values = policy_net(state)
    # The DQN outputs a flattened action index, convert it back to the discrete action (0, 1, or 2)
    action = torch.argmax(q_values).item()
    return action


for ep in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Select an action (0, 1, or 2)
        action = select_action(state, epsilon)

        # Pass the integer action directly to the environment's step function
        next_state, reward, done, info = env.step(action)

        # Ensure 'done' is a boolean to avoid numpy.bool8 error
        done = bool(done)


        # Flatten states for storage in memory
        state_flat = np.array(state).flatten()
        next_state_flat = np.array(next_state).flatten()

        # Store the integer action in memory
        memory.append((state_flat, action, reward, next_state_flat, done))
        state = next_state
        total_reward += reward

        # Train from samples
        if len(memory) > batch_size:
            batch = random.sample(memory, batch_size)
            batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = zip(*batch)

            batch_states = torch.FloatTensor(batch_states)
            batch_actions = torch.LongTensor(batch_actions).unsqueeze(1)
            batch_rewards = torch.FloatTensor(batch_rewards)
            batch_next_states = torch.FloatTensor(batch_next_states)
            batch_dones = torch.FloatTensor(batch_dones)

            q_values = policy_net(batch_states).gather(1, batch_actions).squeeze()
            next_q_values = target_net(batch_next_states).max(1)[0]
            target = batch_rewards + gamma * next_q_values * (1 - batch_dones)

            loss = nn.MSELoss()(q_values, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if done:
            print(f"Episode {ep}, Total reward: {total_reward}")

    # Epsilon decay and target net update
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    if ep % 10 == 0:
        target_net.load_state_dict(policy_net.state_dict())

Episode 0, Total reward: 103
Episode 1, Total reward: -9
Episode 2, Total reward: 39
Episode 3, Total reward: 10
Episode 4, Total reward: -5
Episode 5, Total reward: -6
Episode 6, Total reward: -10
Episode 7, Total reward: -7
Episode 8, Total reward: 79
Episode 9, Total reward: -9
Episode 10, Total reward: -8
Episode 11, Total reward: 48
Episode 12, Total reward: 109
Episode 13, Total reward: 26


/tmp/ipython-input-1154293958.py:60: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch_states = torch.FloatTensor(batch_states)


Episode 14, Total reward: 27
Episode 15, Total reward: 122
Episode 16, Total reward: 50
Episode 17, Total reward: -10
Episode 18, Total reward: -8
Episode 19, Total reward: 28
Episode 20, Total reward: -5
Episode 21, Total reward: -10
Episode 22, Total reward: 20
Episode 23, Total reward: 49
Episode 24, Total reward: -9
Episode 25, Total reward: 110
Episode 26, Total reward: -9
Episode 27, Total reward: -7
Episode 28, Total reward: 127
Episode 29, Total reward: 84
Episode 30, Total reward: -9
Episode 31, Total reward: 55
Episode 32, Total reward: 64
Episode 33, Total reward: 38
Episode 34, Total reward: 75
Episode 35, Total reward: -10
Episode 36, Total reward: 20
Episode 37, Total reward: 60
Episode 38, Total reward: 129
Episode 39, Total reward: -9
Episode 40, Total reward: 25
Episode 41, Total reward: 88
Episode 42, Total reward: -10
Episode 43, Total reward: 115
Episode 44, Total reward: 113
Episode 45, Total reward: 118
Episode 46, Total reward: 102
Episode 47, Total reward: -10
E

In [8]:
torch.save(policy_net.state_dict(), "dqn_minesweeper.pth")
policy_net.load_state_dict(torch.load("dqn_minesweeper.pth"))

<All keys matched successfully>